In [1]:
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt
import os
import glob
import dask.array as da
import datetime as dt

In [2]:
def mkdirs(fsave):
    saveloc=os.path.dirname(fsave)
    if not os.path.exists(saveloc):
        try:
            os.makedirs(saveloc)
        except FileExistsError:
            pass # in case other code running at the same time got to it first
    return

In [3]:
# ind=(mm==(im-1)%12)|(mm==im)|(mm==(im+1)%12) # 3 month groups running mean

In [4]:
ptile=90

In [5]:
years = [1991, 2020]

In [6]:
basepath='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/hindcast/'
fpat=lambda yyyy,mm,lead: f'{basepath}{yyyy}{mm:02}_MSC_CanSIPS-Hindcast_WaterTemp_Sfc_LatLon1.0_P{lead:02}M.grib2'

In [7]:
il=0

In [8]:
dirout = '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/newHindcastMonthly/'

In [9]:
flist=[fpat(yyyy,mm,il) for yyyy in range(years[0],years[1]+1) for mm in range(1,13)]
len(flist)

360

In [10]:
f1=xr.open_mfdataset(flist,engine='cfgrib',combine='nested',concat_dim='time',parallel=True,decode_times=False)#chunks={'number':1,'latitude':-1,'longitude':-1}

In [11]:
f1.sst

<xarray.DataArray 'sst' (time: 360, number: 40, latitude: 180, longitude: 360)> Size: 4GB
dask.array<concatenate, shape=(360, 40, 180, 360), dtype=float32, chunksize=(1, 40, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * number     (number) int64 320B 1 2 3 4 5 6 7 8 9 ... 33 34 35 36 37 38 39 40
  * time       (time) int64 3kB 662688000 665366400 ... 1604188800 1606780800
    surface    float64 8B 0.0
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * longitude  (longitude) float64 3kB 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
Attributes: (12/31)
    GRIB_paramId:                             34
    GRIB_dataType:                            af
    GRIB_numberOfPoints:                      64800
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgd
    ...                                       ...
    GRIB_shortName:                           sst
    GRIB_totalNumber:                         41
    GRIB_units:                               K
    long_name:                                Sea surface temperature
    units:                                    K
    standard_name:                            unknown

In [12]:
f1.number

<xarray.DataArray 'number' (number: 40)> Size: 320B
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40])
Coordinates:
  * number   (number) int64 320B 1 2 3 4 5 6 7 8 9 ... 33 34 35 36 37 38 39 40
    surface  float64 8B 0.0
Attributes:
    long_name:      ensemble member numerical id
    units:          1
    standard_name:  realization

In [13]:
f1.sst

<xarray.DataArray 'sst' (time: 360, number: 40, latitude: 180, longitude: 360)> Size: 4GB
dask.array<concatenate, shape=(360, 40, 180, 360), dtype=float32, chunksize=(1, 40, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * number     (number) int64 320B 1 2 3 4 5 6 7 8 9 ... 33 34 35 36 37 38 39 40
  * time       (time) int64 3kB 662688000 665366400 ... 1604188800 1606780800
    surface    float64 8B 0.0
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * longitude  (longitude) float64 3kB 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
Attributes: (12/31)
    GRIB_paramId:                             34
    GRIB_dataType:                            af
    GRIB_numberOfPoints:                      64800
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgd
    ...                                       ...
    GRIB_shortName:                           sst
    GRIB_totalNumber:                         41
    GRIB_units:                               K
    long_name:                                Sea surface temperature
    units:                                    K
    standard_name:                            unknown

In [14]:
t0=dt.datetime(1970,1,1)
time=np.array([t0 + dt.timedelta(seconds=ii.astype(float)) for ii in f1.time.values])
months=np.array([t1.month-t0.month+(t1.year-t0.year)*12 for t1 in time])

In [15]:
f_out = f'{dirout}sst_HC_l{il}_{years[0]}_{years[1]}.nc'
mkdirs(f_out)

In [16]:
xout=xr.Dataset(data_vars={'start_time':(['S'],f1.time.values),
                        'sst':(['S','M','Y','X'],f1.sst.data)},
            coords=dict(lon=f1.longitude.values,lat=f1.latitude.values,M=f1.number.values,S=months),)
xout.S.attrs['units']='months'
xout.S.attrs['long_name']='Months since 1970-01'


In [17]:
xout

<xarray.Dataset> Size: 4GB
Dimensions:     (S: 360, M: 40, Y: 180, X: 360, lon: 360, lat: 180)
Coordinates:
  * lon         (lon) float64 3kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * lat         (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * M           (M) int64 320B 1 2 3 4 5 6 7 8 9 ... 32 33 34 35 36 37 38 39 40
  * S           (S) int64 3kB 252 253 254 255 256 257 ... 607 608 609 610 611
Dimensions without coordinates: Y, X
Data variables:
    start_time  (S) int64 3kB 662688000 665366400 ... 1604188800 1606780800
    sst         (S, M, Y, X) float32 4GB dask.array<chunksize=(1, 40, 180, 360), meta=np.ndarray>

In [18]:
print(dt.datetime.now())
xout.to_netcdf(f_out,mode='w')
print(dt.datetime.now())

2024-08-14 22:24:23.790041
2024-08-14 22:33:30.065856


In [19]:
f1.close()

In [20]:
print(f_out)

/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/newHindcastMonthly/sst_HC_l0_1991_2020.nc
